In [ ]:
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection  import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from collections import Counter

from keras.models import Sequential
from keras.layers import LSTM, GRU,SimpleRNN
from keras.layers import Dense, Activation, Dropout
from keras.layers import Embedding
from keras.layers import BatchNormalization
from keras.utils import np_utils,pad_sequences
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

import tensorflow as tf

import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from bs4 import BeautifulSoup
import re

%matplotlib inline
import warnings
from tqdm import tqdm
import os
warnings.filterwarnings("ignore")

oneE = OneHotEncoder(sparse=False)

In [ ]:
sentiment = pd.read_csv('Labeled_data.csv')

In [ ]:
sentiment['sentiment'].isnull().sum()

In [ ]:
sentiment['sentiment'].value_counts()

In [ ]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

def preprocess(data):
    preprocessed_reviews = []
    for sentance in tqdm(data):
        sentance = re.sub(r"http\S+", "", sentance)
        sentance = BeautifulSoup(sentance, 'lxml').get_text()
        sentance = decontracted(sentance)
        sentance = re.sub("\S*\d\S*", "", sentance).strip()
        sentance = re.sub('[^A-Za-z]+', ' ', sentance)
        sentance = ' '.join([word for word in sentance.split() if len(word) > 1])
        sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
        preprocessed_reviews.append(sentance.strip())
    return preprocessed_reviews
sentiment_text = preprocess(sentiment['text'].values)

In [ ]:
def sent2Words(preprocess_text):
    i=0
    list_of_sentance=[]
    for sentance in preprocess_text:
        list_of_sentance.append(sentance.split())
    return list_of_sentance
sentiment_sentence = sent2Words(sentiment_text)

# SENTIMENT DATASET TOKENIZATION

In [ ]:
X=sentiment_text
y=sentiment['sentiment'].values

In [ ]:
X = X[:-1]
y = y[:-1]

In [ ]:
import json
def tokenize(data,max_features=20000):
    max_fatures=max_features
    tokenizer = Tokenizer(num_words=max_fatures)
    tokenizer.fit_on_texts(data)
    dictionary = tokenizer.word_index
    with open('wordindex.json', 'w') as dictionary_file:
        json.dump(dictionary, dictionary_file) 
    return tokenizer,dictionary,max_fatures

In [ ]:
tokenizer,dictionary,max_features = tokenize(X)

In [ ]:
def text_to_seq(tokenizer,CodeSnippet,seq=100):
    X = tokenizer.texts_to_sequences(CodeSnippet)
    X = pad_sequences(X,seq)    
    return X

In [ ]:
X = text_to_seq(tokenizer,X)

In [ ]:
def one_hot_encoder(y,inverse=False):
    if inverse:
        Y = oneE.inverse_transform(y)
    else:
        y = np.array(y)
        y = y.reshape(-1,1)
        Y = oneE.fit_transform(y)
    return Y,oneE

In [ ]:
Y,oneE = one_hot_encoder(y)

# **SPLITTING DATASETS**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

## DEEP LEARNING MODELS

In [ ]:
# SIMPLE RNN

In [ ]:
word_index = tokenizer.word_index
max_len = 100

model = Sequential()
model.add(Embedding(len(word_index) + 1,
                 200,
                 input_length=max_len))
model.add(SimpleRNN(100))
model.add(Dense(Y.shape[1], activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(X_train, y_train,validation_data=(X_test,y_test),epochs=10, batch_size=64)

In [ ]:
scores = model.predict(X_test)
one_hot_encoder(scores,inverse=True)

In [ ]:
model.save('SimpleRNN.h5')

In [ ]:
# LSTM WITH WORD EMBEDDINGS

In [ ]:
embeddings_index = {}
f = open('/content/drive/MyDrive/glove.840B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                 300,
                 weights=[embedding_matrix],
                 input_length=max_len,
                 trainable=False))

model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(Y.shape[1], activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(X_train, y_train,validation_data=(X_test,y_test),epochs=10, batch_size=64)

In [ ]:
model.save('models/LSTM.h5')

In [ ]:
# LSTM + GRU

In [ ]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
             300,
             weights=[embedding_matrix],
             input_length=max_len,
             trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(300))
model.add(Dense(Y.shape[1], activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(X_train, y_train,validation_data=(X_test,y_test),epochs=10, batch_size=64)

In [ ]:
# BIDIRECTIONAL LSTM

In [ ]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                 300,
                 weights=[embedding_matrix],
                 input_length=max_len,
                 trainable=False))
model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))
model.add(Dense(Y.shape[1], activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(X_train, y_train,validation_data=(X_test,y_test),epochs=10, batch_size=64)

# BERT MODEL

In [ ]:
X=sentiment_text
y=sentiment['sentiment'].values

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.1,shuffle=True)

In [ ]:
sentiment['text'].apply(lambda x: len(str(x))).max()

In [ ]:
seq_len = 256
batch_size = 32
num_samples = len(X_train)
model_name = 'distilbert-base-uncased-finetuned-sst-2-english'

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

train_tokens = tokenizer(X_train, max_length=seq_len, 
                         truncation=True, padding='max_length', 
                         add_special_tokens=True, return_tensors='np')

# y_train = ytrain['target'].values
labels = np.zeros((num_samples, y_train.max() + 1))
labels[np.arange(num_samples), y_train] = 1
# labels = y_train

dataset = tf.data.Dataset.from_tensor_slices((train_tokens['input_ids'], train_tokens['attention_mask'], labels))

def map_func(input_ids, masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': masks
    }, labels

dataset = dataset.map(map_func)
dataset = dataset.shuffle(10000).batch(batch_size=batch_size, drop_remainder=True)

split = 0.9
size = int((train_tokens['input_ids'].shape[0] // batch_size) * split)

train_ds = dataset.take(size)
val_ds = dataset.skip(size)


In [ ]:
model = TFAutoModel.from_pretrained(model_name)

# Two inputs
input_ids = tf.keras.layers.Input(shape=(seq_len,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(seq_len,), name='attention_mask', dtype='int32')

# Transformer
# embeddings = model.bert(input_ids, attention_mask=mask)[1]
embeddings = model(input_ids, attention_mask=mask)[0]
embeddings = embeddings[:, 0, :]
# Classifier head
x = tf.keras.layers.Dense(512, activation='relu')(embeddings)
# x = tf.keras.layers.Dropout(0.1)(x)
y = tf.keras.layers.Dense(len(set(y)), activation='softmax', name='outputs')(x)

bert_model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

# freeze bert layers
bert_model.layers[2].trainable = False

optimizer = tf.keras.optimizers.Adam(lr=1e-5)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.BinaryAccuracy()

bert_model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

history = bert_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    batch_size=batch_size
)